# search csv

search all files for given header

In [1]:
import sys
sys.path.append('../../')
import glob
import shutil
from pathlib import Path
import os

import pandas as pd
from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR

from scripts.normalize_data import (
    check_duplicate_columns
)

from scripts.shared_utils import (
    log_df
)

In [2]:
clean_data_path = CLEAN_DATA_DIR

metadata_file = OUTPUT_DIR/'metadata'/'NOAA'/'noaa_dsdp_files.csv' 
metadata2_file = OUTPUT_DIR/'metadata'/'NOAA'/'noaa_janus_files.csv' 
metadata3_file = OUTPUT_DIR/'metadata'/'NOAA'/'noaa_janus_iodp_files.csv' 

all_columns_file = OUTPUT_DIR/'tmp/all_NOAA_taxa_columns.csv'


## create a file with all columns  in LIMS taxa

In [51]:
data = []

In [52]:
metadata = pd.read_csv(metadata_file)
log_df(metadata)

(4477, 5)


,path,type,taxon_group,expedition,site
0,NOAA/DSDP_core_data/61/462/radiolar.csv,taxa,radiolarians,61,462
1,NOAA/DSDP_core_data/61/462/ageprof.csv,age,NaN,61,462
2,NOAA/DSDP_core_data/61/462/b_forams.csv,taxa,benthic_forams,61,462
3,NOAA/DSDP_core_data/61/462/p_forams.csv,taxa,planktic_forams,61,462
4,NOAA/DSDP_core_data/61/462/hr_desc.csv,hard_rock,NaN,61,462


In [53]:
metadata['type'].unique()

array(['taxa', 'age', 'hard_rock', 'lithology'], dtype=object)

In [54]:
for index, row in metadata.iterrows():
    if row['type'] != 'taxa':
        continue
        
    df = pd.read_csv(clean_data_path/row['path'], dtype=str)
    taxa = df['fossil'].unique()
    for taxon in taxa:
        data.append({"path": row['path'], "column": taxon})

In [55]:
metadata2 = pd.read_csv(metadata2_file)
log_df(metadata2)

(325, 4)


,path,type,expedition,site
0,NOAA/Janus_core_data/101/626a/sed_lith_101_626...,lithology,101,626a
1,NOAA/Janus_core_data/101/626b/sed_lith_101_626...,lithology,101,626b
2,NOAA/Janus_core_data/101/626c/sed_lith_101_626...,lithology,101,626c
3,NOAA/Janus_core_data/101/626d/sed_lith_101_626...,lithology,101,626d
4,NOAA/Janus_core_data/101/627a/sed_lith_101_627...,lithology,101,627a


In [56]:
metadata2['type'].unique()

array(['lithology'], dtype=object)

In [57]:
metadata3 = pd.read_csv(metadata3_file)
log_df(metadata3)

(2481, 5)


,path,type,expedition,site,taxon_group
0,NOAA/JanusIODP_paleo_agemodel/paleontology/ran...,taxa,101,626,benthic_forams
1,NOAA/JanusIODP_paleo_agemodel/paleontology/ran...,taxa,101,626,benthic_forams
2,NOAA/JanusIODP_paleo_agemodel/paleontology/ran...,taxa,101,626,nannofossils
3,NOAA/JanusIODP_paleo_agemodel/paleontology/ran...,taxa,101,626,nannofossils
4,NOAA/JanusIODP_paleo_agemodel/paleontology/ran...,taxa,101,626,planktic_forams


In [58]:
metadata3['type'].unique()

array(['taxa', 'age'], dtype=object)

In [59]:

for index, row in metadata3.iterrows():
    if row['type'] != 'taxa':
        continue
        
    df = pd.read_csv(clean_data_path/row['path'], dtype=str)
    df.dropna(axis="columns", how='all', inplace=True)
    for col in df.columns:
        data.append({"path": row['path'], "column": col})

In [61]:
all_df = pd.DataFrame(data)
all_df.drop_duplicates(inplace=True)
log_df(all_df)
# 188702

(188702, 2)


,path,column
0,NOAA/DSDP_core_data/61/462/radiolar.csv,Stichocorys peregrina
1,NOAA/DSDP_core_data/61/462/radiolar.csv,Artophormis gracilis
2,NOAA/DSDP_core_data/61/462/radiolar.csv,Artostrobium doliolum
3,NOAA/DSDP_core_data/61/462/radiolar.csv,Lithopera renzae
4,NOAA/DSDP_core_data/61/462/radiolar.csv,Solenospheara omnitubus


In [62]:
all_df.to_csv(all_columns_file)

## search for column

In [24]:
def find_column_value(all_columns_path, column, copy_files=False):
    all_df = pd.read_csv(all_columns_path, dtype=str)
    files = list(all_df[all_df['column'] == column]['path'].unique())
    
    print(f'"{column}" found in {len(files)} files\n')

    for file in files:
        path = CLEAN_DATA_DIR/file
        df = pd.read_csv(path, dtype=str, usecols=[column])
        df[column].dropna(inplace=True)
        
        print(path, '\n')
        if len(df[column].unique()) > 0:
            print(' | '.join(df[column].fillna('').unique()))
            print('----')
        
        if copy_files:
            temp_path = Path('../','NOAA', 'tmp', file)
            if not os.path.exists(temp_path.parent):
                os.makedirs(temp_path.parent)
            shutil.copy(path, temp_path.parent)


In [25]:
column = 'Falsebrioid Falsebrioid'

find_column_value(all_columns_file, column, True)

"Falsebrioid Falsebrioid" found in 3 files

../../output/cleaned_data/NOAA/JanusIODP_paleo_agemodel/paleontology/range_tables/104/642/HOLE_C/Silicoflagellates_Ebridians_Actiniscidians.csv 

 | 13.0%                   |                         | 20.0%                   | 12.0%                   | 3.0%                    | 1.0%                    | 11.0%                   | 8.0%                    | 10.0%                   | 4.0%                    | 5.0%                    | 9.0%                    | 14.0%                   | 15.0%                   | 7.0%                    | 2.0%                    | 24.0%                   | 28.0%                   | 36.0%                   | 34.0%                   | 16.0%                  
----
../../output/cleaned_data/NOAA/JanusIODP_paleo_agemodel/paleontology/range_tables/104/642/HOLE_D/Silicoflagellates_Ebridians_Actiniscidians.csv 

 |                         | 10.0%                   | 9.0%                    | 5.0%                    | 8.0% 